In [1]:
import json
import shutil
from dataclasses import dataclass
from pathlib import Path
from typing import Optional

import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sqlalchemy import select, BinaryExpression, func

from databases.db_models import DBPost
from databases.external import DBConfig, SQliteConnection
from databases.platform_db_mgmt import PlatformDB
from src.const import BASE_DATA_PATH

stats_copy_path = BASE_DATA_PATH / "stats_copy.sqlite"
from dataclasses import field
from collections import Counter
from datetime import date
from databases.external import ClientTaskConfig
from calendar import monthrange
from datetime import datetime, time
import pytz

In [2]:
def make_stats_copy(db_path: Path):
    shutil.copy(db_path, stats_copy_path)


make_stats_copy(BASE_DATA_PATH / "twitter_20_01_2025_backup.sqlite")

# old..
platform_db = PlatformDB("twitter", DBConfig(db_connection=SQliteConnection(db_path=stats_copy_path)))

db = platform_db.db_mgmt


2025-01-29 15:05:20,180 - ...Databases.databases.db_mgmt - DEBUG - creating db engine with sqlite:////home/rsoleyma/projects/platforms-clients/data/stats_copy.sqlite


In [3]:
def get_all_posts_sorted(platform: str,
                         conditions: Optional[BinaryExpression | list[BinaryExpression]] = None
                         ) -> list[DBPost]:
    with db.get_session() as session:
        # Start with a base query
        query = select(DBPost)

        # Platform is required
        query = query.where(DBPost.platform == platform)

        if conditions is not None:
            if isinstance(conditions, list):
                for condition in conditions:
                    query = query.where(condition)
            else:
                query = query.where(conditions)

        query.order_by(DBPost.date_created)
        # Execute the query and return the results
        result = session.execute(query)
        for post in result.scalars():
            yield post.model()


In [4]:
from databases.model_conversion import PostModel
from typing import Iterator

posts: Iterator[PostModel] = get_all_posts_sorted("twitter")
labeled_posts: list[PostModel] = []
for post in posts:
    labels = post.metadata_content_model.labels
    if labels:
        labeled_posts.append(post)

/tmp/ipykernel_12727/2120273466.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) metadata_content_model. (just use metadata_content)
  labels = post.metadata_content_model.labels


In [5]:
all_datetimes: list[datetime] = []

for post in labeled_posts:
    all_datetimes.append(post.date_created)

all_datetimes.sort()

In [14]:
from datetime import timedelta


@dataclass(frozen=True)
class PostRange:
    year: int
    month: int
    day: int
    hour: int


def create_all_dt_tuples(start: datetime, end: datetime, exclude_end: Optional[bool] = True) -> list[PostRange]:
    current = start
    all_ranges: list[PostRange] = []
    while current <= end:
        all_ranges.append(PostRange(year=current.year, month=current.month, day=current.day, hour=current.hour))
        current += timedelta(hours=1)
    if exclude_end:
        all_ranges.pop(-1)
    return all_ranges

required_times = create_all_dt_tuples(datetime(2023, 1, 1), datetime(2023, 5, 1), exclude_end=True)
time_counter = Counter({t: 0 for t in required_times})

In [15]:
for d in all_datetimes:
    pr = PostRange(year=d.year, month=d.month, day=d.day, hour=d.hour)
    time_counter[pr] += 1

In [16]:
for t, c in time_counter.items():
    if c != 1:
        print(f"{t}: {c}")

PostRange(year=2023, month=1, day=1, hour=23): 0
PostRange(year=2023, month=1, day=2, hour=0): 0
PostRange(year=2023, month=1, day=2, hour=2): 2
PostRange(year=2023, month=1, day=2, hour=5): 0
PostRange(year=2023, month=1, day=2, hour=6): 0
PostRange(year=2023, month=1, day=2, hour=7): 3
PostRange(year=2023, month=1, day=2, hour=9): 0
PostRange(year=2023, month=1, day=2, hour=11): 0
PostRange(year=2023, month=1, day=2, hour=12): 0
PostRange(year=2023, month=1, day=2, hour=13): 2
PostRange(year=2023, month=1, day=2, hour=15): 0
PostRange(year=2023, month=1, day=2, hour=17): 2
PostRange(year=2023, month=1, day=2, hour=18): 0
PostRange(year=2023, month=1, day=2, hour=19): 2
PostRange(year=2023, month=1, day=2, hour=21): 2
PostRange(year=2023, month=1, day=2, hour=22): 0
PostRange(year=2023, month=1, day=2, hour=23): 3
PostRange(year=2023, month=1, day=3, hour=0): 0
PostRange(year=2023, month=1, day=3, hour=2): 0
PostRange(year=2023, month=1, day=3, hour=4): 0
PostRange(year=2023, month=1,

In [21]:
def search_md_c():
    with db.get_session() as session:
            query = select(DBPost)
            # Platform is required
            query = query.where(DBPost.platform == "twitter")
            query = query.where(DBPost.metadata_content["labels"].contains("test_selection"))
            result = session.execute(query)
            for post in result.scalars():
                yield post.model()

for a in search_md_c():
    print(a.metadata_content.labels)
    break

['test_selection']
